In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os, glob

### I/O

In [2]:
# raw data from model runs
runs_dir = 'M:/Data/Urban/BAUS/visualization_design/data/data_raw_runs'

# data ready for visualization
viz_dir = 'M:/Data/Urban/BAUS/visualization_design/data/data_viz_ready'
viz_csv_dir = os.path.join(viz_dir, 'csv')
viz_spatial_dir = os.path.join(viz_dir, 'spatial')
viz_crosswalks_dir = os.path.join(viz_dir, 'crosswalks')


### create growth geography spatial files

In [4]:
%%time
# parcels_geography file
raw_df = pd.read_csv(
    os.path.join(runs_dir, 'old_inputs', '2021_02_25_parcels_geography.csv'),
    usecols = ['PARCEL_ID', 'juris', 
               'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
               'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat',
               'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
               'eir_exp2020_id', 'eirzoningmodcat'])

# juris-county crosswalk
juris_county_df = pd.read_csv(
    os.path.join(viz_dir, 'crosswalks', 'juris_county_id.csv'),
    usecols = ['juris_name_full', 'county_name', 'baus_output_juris_name', 'shapefile_juris_name'])
juris_county_df.rename(columns={'juris_name_full': 'juris'}, inplace=True)

CPU times: total: 6.55 s
Wall time: 3min 33s


In [6]:
for i in ['juris', 
               'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
               'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat',
               'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
               'eir_exp2020_id', 'eirzoningmodcat']:
    raw_df[i].fillna('NA', inplace=True)

In [5]:
%%time
# parcel spatial file
p10_geo_file = os.path.join(r'M:\Data\GIS layers\UrbanSim smelt\p10_EIR\p10_parcels.shp')
p10_geo = gpd.read_file(p10_geo_file)
p10_geo = p10_geo[['PARCEL_ID', 'geometry']]
p10_geo['PARCEL_ID'] = p10_geo['PARCEL_ID'].apply(lambda x: int(round(x)))

CPU times: total: 5min 9s
Wall time: 36min 27s


In [7]:
# attach county name
p10_df = raw_df.merge(juris_county_df, on='juris', how='left')

# add geometry to parcels
p10_geographies = pd.merge(
    p10_geo[['PARCEL_ID', 'geometry']],
    p10_df,
    on='PARCEL_ID', how='left')

In [8]:
print(list(p10_geographies))

['PARCEL_ID', 'geometry', 'juris', 'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id', 'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat', 'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id', 'eir_exp2020_id', 'eirzoningmodcat', 'county_name', 'baus_output_juris_name', 'shapefile_juris_name']


#### zoningmodcat spatial file

In [11]:
%%time
# scenario 24, 25: zoning policy based on "fbpzoningmodcat"
p10_fbpzoningmodcat = p10_geographies[[
    'PARCEL_ID', 'geometry', 'juris', 'county_name', 'fbpzoningmodcat']]

# dissolve based on 'fbpzoningmodcat'
fbpzoningmodcat_gdf = p10_fbpzoningmodcat.dissolve(by='fbpzoningmodcat')
fbpzoningmodcat_gdf = fbpzoningmodcat_gdf.reset_index()[['fbpzoningmodcat', 'geometry']]

# make column names scenario agnostic
if 'fbpzoningmodcat' in list(fbpzoningmodcat_gdf):
    fbpzoningmodcat_gdf.rename(columns = {'fbpzoningmodcat': 'zmodscat'}, inplace=True)
    fbpzoningmodcat_gdf['geo_type'] = 'zoningmodcat_fbp'

# some stats
print(fbpzoningmodcat_gdf.shape[0])
print(fbpzoningmodcat_gdf.zmodscat.nunique())
print(fbpzoningmodcat_gdf.zmodscat.unique())

display(fbpzoningmodcat_gdf.head())

2970
2970
['alamedaGGNADISNAinNA' 'alamedaGGNADISNAinsfd' 'alamedaGGNAHRADISNAinNA'
 ... 'yountvilleNANAHRADISNAinNA' 'yountvilleNANAHRADISNAinsfd'
 'yountvilleNANAHRADISNAoutNA']


,zmodscat,geometry,geo_type
0,alamedaGGNADISNAinNA,"MULTIPOLYGON (((562769.181 4180941.536, 562769...",zoningmodcat_fbp
1,alamedaGGNADISNAinsfd,"MULTIPOLYGON (((562526.655 4180936.283, 562524...",zoningmodcat_fbp
2,alamedaGGNAHRADISNAinNA,"MULTIPOLYGON (((564408.266 4179585.190, 564408...",zoningmodcat_fbp
3,alamedaGGNAHRADISNAinsfd,"MULTIPOLYGON (((564442.572 4179674.308, 564448...",zoningmodcat_fbp
4,alamedaGGNAHRANAinNA,"MULTIPOLYGON (((565171.573 4179235.296, 565118...",zoningmodcat_fbp


CPU times: total: 7min 42s
Wall time: 7min 45s


In [16]:
fbpzoningmodcat_gdf

,zmodscat,geometry,geo_type
0,alamedaGGNADISNAinNA,"MULTIPOLYGON (((562769.181 4180941.536, 562769...",zoningmodcat_fbp
1,alamedaGGNADISNAinsfd,"MULTIPOLYGON (((562526.655 4180936.283, 562524...",zoningmodcat_fbp
2,alamedaGGNAHRADISNAinNA,"MULTIPOLYGON (((564408.266 4179585.190, 564408...",zoningmodcat_fbp
3,alamedaGGNAHRADISNAinsfd,"MULTIPOLYGON (((564442.572 4179674.308, 564448...",zoningmodcat_fbp
4,alamedaGGNAHRANAinNA,"MULTIPOLYGON (((565171.573 4179235.296, 565118...",zoningmodcat_fbp
...,...,...,...
2965,yountvilleNANADISNAinsfd,"MULTIPOLYGON (((555999.934 4250425.533, 556010...",zoningmodcat_fbp
2966,yountvilleNANADISNAoutNA,"MULTIPOLYGON (((554508.062 4247992.667, 553977...",zoningmodcat_fbp
2967,yountvilleNANAHRADISNAinNA,"MULTIPOLYGON (((555478.997 4250828.204, 555504...",zoningmodcat_fbp
2968,yountvilleNANAHRADISNAinsfd,"MULTIPOLYGON (((555373.859 4250961.319, 555387...",zoningmodcat_fbp


In [17]:
# write out
fbpzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'zoningmodcat_fbp.shp'))

2970

In [13]:
%%time
# scenario 28: zoning policy based on "eirzoningmodcat"
p10_eirzoningmodcat = p10_geographies[[
    'PARCEL_ID', 'geometry', 'juris', 'county_name', 'eirzoningmodcat']]

# dissolve based on 'eirzoningmodcat'
eirzoningmodcat_gdf = p10_eirzoningmodcat.dissolve(by='eirzoningmodcat')
eirzoningmodcat_gdf = eirzoningmodcat_gdf.reset_index()[['eirzoningmodcat', 'geometry']]

# make column names scenario agnostic
eirzoningmodcat_gdf.rename(columns = {'eirzoningmodcat': 'zmodscat'}, inplace=True)
eirzoningmodcat_gdf['geo_type'] = 'zoningmodcat_eir'

# some stats
print(eirzoningmodcat_gdf.shape[0])
print(eirzoningmodcat_gdf.zmodscat.nunique())
print(eirzoningmodcat_gdf.zmodscat.unique())

display(eirzoningmodcat_gdf.head())

3535
3535
['alamedaGGNADISNANAinNA' 'alamedaGGNADISNANAinres'
 'alamedaGGNAHRADISNANAinNA' ... 'yountvilleNANAHRADISNANAinNA'
 'yountvilleNANAHRADISNANAinres' 'yountvilleNANAHRADISNANAoutNA']


,zmodscat,geometry,geo_type
0,alamedaGGNADISNANAinNA,"POLYGON ((562749.669 4181233.776, 562748.717 4...",zoningmodcat_eir
1,alamedaGGNADISNANAinres,"MULTIPOLYGON (((562526.655 4180936.283, 562524...",zoningmodcat_eir
2,alamedaGGNAHRADISNANAinNA,"MULTIPOLYGON (((564321.546 4179921.117, 564286...",zoningmodcat_eir
3,alamedaGGNAHRADISNANAinres,"MULTIPOLYGON (((564408.077 4179585.287, 564407...",zoningmodcat_eir
4,alamedaGGNAHRANANAinNA,"MULTIPOLYGON (((565739.921 4178974.835, 565722...",zoningmodcat_eir


CPU times: total: 6min 42s
Wall time: 6min 43s


In [20]:
# write out
eirzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'zoningmodcat_eir.shp'))

#### fbpchcat spatial file (for inclusionary zoning strategy)

In [14]:
%%time
# scenario 24, 28: inclusionary zoning based on 'fbpchcat'
p10_fbpchcat = p10_geographies[['PARCEL_ID', 'geometry', 'fbpchcat']]

# dissolve on 'fbpchcat'
fbpchcat_gdf = p10_fbpchcat.dissolve(by='fbpchcat')
fbpchcat_gdf = fbpchcat_gdf.reset_index()[['fbpchcat', 'geometry']]
print(fbpchcat_gdf.shape[0])
display(fbpchcat_gdf.head())

fbpchcat_gdf.columns = ['fbpchcat', 'geometry']
fbpchcat_gdf['geo_type'] = 'fbpchcat'

208


,fbpchcat,geometry
0,GGNADISNAexp0,"MULTIPOLYGON (((517487.627 4261506.715, 517473..."
1,GGNADISNAexp2_np,"POLYGON ((590580.239 4241349.472, 590546.129 4..."
2,GGNADISNAin,"MULTIPOLYGON (((498557.397 4292316.379, 498583..."
3,GGNADISNAinun,"MULTIPOLYGON (((523001.584 4251701.413, 522997..."
4,GGNADISNAout,"MULTIPOLYGON (((543866.860 4202404.878, 543867..."


CPU times: total: 13min 36s
Wall time: 13min 39s


In [18]:
fbpchcat_gdf

,fbpchcat,geometry,geo_type
0,GGNADISNAexp0,"MULTIPOLYGON (((517487.627 4261506.715, 517473...",fbpchcat
1,GGNADISNAexp2_np,"POLYGON ((590580.239 4241349.472, 590546.129 4...",fbpchcat
2,GGNADISNAin,"MULTIPOLYGON (((498557.397 4292316.379, 498583...",fbpchcat
3,GGNADISNAinun,"MULTIPOLYGON (((523001.584 4251701.413, 522997...",fbpchcat
4,GGNADISNAout,"MULTIPOLYGON (((543866.860 4202404.878, 543867...",fbpchcat
...,...,...,...
203,NAtra3NANAinun,"MULTIPOLYGON (((545521.122 4171145.565, 545501...",fbpchcat
204,NAtra3NANAout,"MULTIPOLYGON (((542747.698 4181388.946, 542746...",fbpchcat
205,NAtra3NAppain,"MULTIPOLYGON (((552324.234 4176612.356, 552299...",fbpchcat
206,NAtra3NAppainun,"POLYGON ((605336.270 4170626.590, 605336.278 4...",fbpchcat


In [19]:
# write out
fbpchcat_gdf.to_file(os.path.join(viz_spatial_dir, 'fbpchcat_gdf.shp'))

In [20]:
# scenario 25: jurisdiction, using existing juris shapefile

#### non-spatial table with key growth geographies tagging

In [76]:
# FBP growth geographies

# parcel-level
parcel_geo_fbp = p10_geographies[['PARCEL_ID', 'juris', 'baus_output_juris_name', 'shapefile_juris_name', 'county_name',
                                  'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
                                  'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat']]

In [77]:
# get fbpzoningmodcat-level growth geography attributes
fbpzoningmodcat_attrs = parcel_geo_fbp[['shapefile_juris_name', 'county_name',
                                        'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
                                        'fbp_exp2020_id', 'fbpzoningmodcat']].drop_duplicates()
print(fbpzoningmodcat_attrs.shape[0])
print(fbpzoningmodcat_attrs.shape[0] == fbpzoningmodcat_gdf.shape[0])

display(fbpzoningmodcat_attrs.head())

2970
True


,shapefile_juris_name,county_name,fbp_gg_id,fbp_tra_id,fbp_sesit_id,fbp_ppa_id,fbp_exp2020_id,fbpzoningmodcat
0,Livermore,Alameda,NA,NA,HRADIS,NA,in,livermoreNANAHRADISNAinNA
1,Livermore,Alameda,NA,tra3,DIS,NA,in,livermoreNAtra3DISNAinNA
2,Hayward,Alameda,NA,NA,NA,NA,in,haywardNANANANAinNA
3,Unincorporated Sonoma,Sonoma,NA,NA,HRADIS,NA,out,unincorporated_sonomaNANAHRADISNAoutNA
4,Fremont,Alameda,NA,NA,HRADIS,NA,in,fremontNANAHRADISNAinNA


In [78]:
# make column names scenario agnostic
fbpzoningmodcat_attrs.rename(columns = {'fbpzoningmodcat': 'zoningmodcat',
                                        'fbp_gg_id': 'gg_id',
                                        'fbp_tra_id': 'tra_id',
                                        'fbp_sesit_id': 'sesit_id',
                                        'fbp_ppa_id': 'ppa_id',
                                        'fbp_exp2020_id': 'exp2020_id'}, inplace=True)
fbpzoningmodcat_attrs['geo_type'] = 'zoningmodcat_fbp'

In [79]:
fbpzoningmodcat_attrs.head()

,shapefile_juris_name,county_name,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,zoningmodcat,geo_type
0,Livermore,Alameda,NA,NA,HRADIS,NA,in,livermoreNANAHRADISNAinNA,zoningmodcat_fbp
1,Livermore,Alameda,NA,tra3,DIS,NA,in,livermoreNAtra3DISNAinNA,zoningmodcat_fbp
2,Hayward,Alameda,NA,NA,NA,NA,in,haywardNANANANAinNA,zoningmodcat_fbp
3,Unincorporated Sonoma,Sonoma,NA,NA,HRADIS,NA,out,unincorporated_sonomaNANAHRADISNAoutNA,zoningmodcat_fbp
4,Fremont,Alameda,NA,NA,HRADIS,NA,in,fremontNANAHRADISNAinNA,zoningmodcat_fbp


In [80]:
fbpzoningmodcat_attrs['county_name'].unique()

array(['Alameda', 'Sonoma', 'Contra Costa', 'Napa', 'San Francisco',
       'San Mateo', 'Marin', 'Santa Clara', 'Solano'], dtype=object)

In [81]:
# EIR growth geographies

# parcel-level
parcel_geo_eir = p10_geographies[['PARCEL_ID', 'juris', 'baus_output_juris_name', 'shapefile_juris_name', 'county_name',
                                  'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
                                  'eir_exp2020_id', 'eirzoningmodcat']]

In [82]:
# get fbpzoningmodcat-level growth geography attributes
eirzoningmodcat_attrs = parcel_geo_eir[['shapefile_juris_name', 'county_name',
                                        'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
                                        'eir_exp2020_id', 'eirzoningmodcat']].drop_duplicates()
print(eirzoningmodcat_attrs.shape[0])
print(eirzoningmodcat_attrs.shape[0] == eirzoningmodcat_gdf.shape[0])

display(eirzoningmodcat_attrs)

3535
True


,shapefile_juris_name,county_name,eir_gg_id,eir_tra_id,eir_sesit_id,eir_coc_id,eir_ppa_id,eir_exp2020_id,eirzoningmodcat
0,Livermore,Alameda,NA,NA,HRADIS,NA,NA,in,livermoreNANAHRADISNANAinNA
1,Livermore,Alameda,NA,tra3,DIS,NA,NA,in,livermoreNAtra3DISNANAinNA
2,Hayward,Alameda,NA,NA,NA,NA,NA,in,haywardNANANANANAinres
3,Unincorporated Sonoma,Sonoma,NA,NA,HRADIS,NA,NA,out,unincorporated_sonomaNANAHRADISNANAoutNA
4,Fremont,Alameda,NA,NA,HRADIS,NA,NA,in,fremontNANAHRADISNANAinNA
...,...,...,...,...,...,...,...,...,...
1936091,Cloverdale,Sonoma,NA,NA,NA,NA,NA,out,cloverdaleNANANANANAoutNA
1940482,Campbell,Santa Clara,NA,tra2c,HRA,NA,NA,in,campbellNAtra2cHRANANAinNA
1943981,Napa,Napa,NA,NA,NA,NA,NA,exp1,napaNANANANANAexp1res
1944180,Unincorporated Napa,Napa,NA,NA,NA,NA,ppa,inun,unincorporated_napaNANANANAppainunNA


In [83]:
# make column names scenario agnostic
eirzoningmodcat_attrs.rename(columns = {'eirzoningmodcat': 'zoningmodcat',
                                        'eir_gg_id': 'gg_id',
                                        'eir_tra_id': 'tra_id',
                                        'eir_sesit_id': 'sesit_id',
                                        'eir_coc_id': 'coc_id',
                                        'eir_ppa_id': 'ppa_id',
                                        'eir_exp2020_id': 'exp2020_id'}, inplace=True)
eirzoningmodcat_attrs['geo_type'] = 'zoningmodcat_eir'

#### simplify the growth geography categories for easier mapping

In [84]:
# fbpzoningmodcat_attrs['tra_id_cat'] = 'non-TRA'
# fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['tra_id'] != 'NA', 'tra_id_cat'] = 'TRA'

# fbpzoningmodcat_attrs['hra_id_cat'] = 'non-HRA'
# fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS']), 'hra_id_cat'] = 'HRA'

fbpzoningmodcat_attrs['simpler_geo_cat'] = ''
fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['gg_id'] == 'NA', 'simpler_geo_cat'] = 'outside GG'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / Transit-Rich and High-Resource'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])), 'simpler_geo_cat'] = 'GG / Transit-Rich Outside HRA'


fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / High-Resource Outside TRA'


fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (fbpzoningmodcat_attrs['ppa_id'] == 'ppa'), 'simpler_geo_cat'] = 'GG / Priority Production Area'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (fbpzoningmodcat_attrs['ppa_id'] == 'NA'), 'simpler_geo_cat'] = 'GG / others'

In [85]:
print(fbpzoningmodcat_attrs['simpler_geo_cat'].value_counts())

outside GG                             1838
GG / Transit-Rich Outside HRA           395
GG / Transit-Rich and High-Resource     319
GG / others                             207
GG / High-Resource Outside TRA          206
GG / Priority Production Area             5
Name: simpler_geo_cat, dtype: int64


In [86]:
# eirzoningmodcat_attrs['tra_id_cat'] = 'non-TRA'
# eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['tra_id'] != 'NA', 'tra_id_cat'] = 'TRA'

# eirzoningmodcat_attrs['hra_id_cat'] = 'non-HRA'
# eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS']), 'hra_id_cat'] = 'HRA'

eirzoningmodcat_attrs['simpler_geo_cat'] = ''
eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['gg_id'] == 'NA', 'simpler_geo_cat'] = 'outside GG'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / Transit-Rich and High-Resource'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])), 'simpler_geo_cat'] = 'GG / Transit-Rich Outside HRA'


eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / High-Resource Outside TRA'


eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (eirzoningmodcat_attrs['ppa_id'] == 'ppa'), 'simpler_geo_cat'] = 'GG / Priority Production Area'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (eirzoningmodcat_attrs['ppa_id'] == 'NA'), 'simpler_geo_cat'] = 'GG / others'

In [87]:
print(eirzoningmodcat_attrs['simpler_geo_cat'].value_counts())

outside GG                             2159
GG / Transit-Rich Outside HRA           559
GG / Transit-Rich and High-Resource     337
GG / others                             273
GG / High-Resource Outside TRA          202
GG / Priority Production Area             5
Name: simpler_geo_cat, dtype: int64


#### export

In [88]:
sesit_recode = {
    'HRADIS': 'hradis',
    'DIS': 'dis',
    'HRA': 'hra'
}

In [89]:
fbpzoningmodcat_attrs['sesit_id'] = fbpzoningmodcat_attrs['sesit_id'].map(sesit_recode)

fbpzoningmodcat_attrs.to_csv(os.path.join(viz_crosswalks_dir, 'fbpzoningmodcat_attrs.csv'), index=False)

In [90]:
eirzoningmodcat_attrs['sesit_id'] = eirzoningmodcat_attrs['sesit_id'].map(sesit_recode)
eirzoningmodcat_attrs.to_csv(os.path.join(viz_crosswalks_dir, 'eirzoningmodcat_attrs.csv'), index=False)